<a href="https://colab.research.google.com/github/dbarrau/TensorFlow_training/blob/main/00_tensorflow_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In this notebook, we're going to cover some of the most fundamental concepts of tensors using TensorFlow.

More specifically, we're going to cover:
* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & NumPy
* Using @tf.function (a way to speed up regular python functions)
* Using GPUs with TensorFlow (or TPUs)
* Exercise to try for yourself

## Introduction to Tensors

In [ ]:
# Import TensorFlow

import tensorflow as tf
print(tf.__version__)

2.8.2


In [ ]:
# Create tensors with tf.constant()

scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [ ]:
# Check the number of dimensions of a tensor (ndim stands for number of dimensions)

scalar.ndim

0

In [ ]:
# Create a vector
vector = tf.constant([10,10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [ ]:
# check the dimension of our vector
vector.ndim

1

In [ ]:
# Create a matrix (has more than 1 dimension)

matrix = tf.constant([[10,7],
                      [7,10]])
matrix
matrix.ndim

2

In [ ]:
# Create another matrix

another_matrix = tf.constant([[10.,7.],
                              [3.,2.],
                              [8.,9.]], dtype=tf.float16)
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [ ]:
# Let's create a tensor

tensor = tf.constant([[[1, 2, 3],
                       [4, 5, 6,]],
                      [[7, 8, 9],
                       [10, 11, 12]],
                      [[13, 14, 15],
                       [16, 17, 18]]])

print(tensor)
print(tensor.ndim)

tf.Tensor(
[[[ 1  2  3]
  [ 4  5  6]]

 [[ 7  8  9]
  [10 11 12]]

 [[13 14 15]
  [16 17 18]]], shape=(3, 2, 3), dtype=int32)
3


What we've created so far:
* Scalar: A single number
* Vector: a number with direction (e.g wind speed and direction)
* Matrix: A 2-dimensional array of numbers
* tensor: An n-dimensional array of numbers (when n can be any number, a 0-dimensional tensor is a scalar, a 1-dimensional tensor is a vector)



### Creating tensors with `tf.Variable()`


In [ ]:
# Create the same tensor with tf.Variable() as above

changeable_tensor = tf.Variable([10,7])
unchangeable_tensor = tf.constant([10,7])
changeable_tensor,unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [ ]:
# Let's try change one of the elements in our changeale tensor


#changeable_tensor[0] = 7

In [ ]:
# How about we try .assign()

changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [ ]:
# let's try with the unchangeable tensor

#unchangeable_tensor[0].assign(7)
#unchangeable_tensor[0] = 7

Some tensors are not meant to be changed. This is why we have to make a proper decision on declaring a tensor as a variable or as a constant. In deep learning models, tensor variables are weights and biases, while features and observations are constant.

### Creating random tensors

Random tensors are tensors of some arbitrary size which contain random numbers. Random tensors are useful to initialize weights for training a deep learning model.

In [ ]:
# Create two random (but the same) tensors

random_1 = tf.random.Generator.from_seed(42) # set seed for reproducibility
random_1 = random_1.normal(shape=(3,2))
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3,2))

random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193765, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193765, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

When creating random numbers, they appear random but actually they are not. these are seudo-random numbers and the "seudo" prefix comes from taking a seed and generating values according to the seed.
Assigning random values to seeds would more likely create random numbers.

### Shuffle the order of elements in a tensor

In [ ]:
# Shuffle a tensor (valuable for when we want to shuffle our data so the inherent order doesn't affect learning)

not_shuffled = tf.constant([[10,7],
                            [3,4],
                           [2, 5]])
not_shuffled.ndim

# shuffle non-shuffled tensor
tf.random.set_seed(42)
tf.random.shuffle(not_shuffled, seed = 21)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 3,  4],
       [ 2,  5],
       [10,  7]], dtype=int32)>

the shuffle method occurs within the first dimension of a tensor. If we have a dataset whose first observations are biased into one category, then it is convenient to shuffle the sample. To clarify, let's say we want to classify plant images as coriander or Parsley. If the training set has so to say six thousand images, and the first four thousand are coriander, then we have an issue. The model will learn about Coriander but nothing about Parsley until they meet with the observations with Parsley. This issue makes shuffling the tensor something very convenient.

With this, rows are shuffled and not columns, as it makes no significance to shuffle columns.

the parameter `seed` will not fix the shuffle and therefore we need `tf.random.set_seed()` 

> rule 4: If we want to shuffle tensors to be in the same order, we have to use the global level random seed as well as the operation level random seed

In [ ]:
tf.random.set_seed(42) # Global level random seed

random_t1 = tf.random.Generator.from_seed(32) # Operation level random seed
random_t1 = random_t1.normal(shape=(4,4))
shuffled_t1 = tf.random.shuffle(random_t1, seed=20)

print(random_t1, shuffled_t1, random_t1 == shuffled_t1)

random_t2 = tf.random.Generator.from_seed(14)
random_t2 = random_t2.normal(shape=(4,4))
shuffled_t2 = tf.random.shuffle(random_t2, seed=23)

print(random_t2, shuffled_t2, random_t2 == shuffled_t2)

tf.Tensor(
[[ 0.7901182   1.585549    0.4356279   0.23645182]
 [-0.15898712  1.302304    0.959224    0.85874265]
 [-1.518177    1.4020647   1.5570306  -0.96762174]
 [ 0.49529105 -0.648484   -1.8700892   2.7830641 ]], shape=(4, 4), dtype=float32) tf.Tensor(
[[-0.15898712  1.302304    0.959224    0.85874265]
 [-1.518177    1.4020647   1.5570306  -0.96762174]
 [ 0.49529105 -0.648484   -1.8700892   2.7830641 ]
 [ 0.7901182   1.585549    0.4356279   0.23645182]], shape=(4, 4), dtype=float32) tf.Tensor(
[[False False False False]
 [False False False False]
 [False False False False]
 [False False False False]], shape=(4, 4), dtype=bool)
tf.Tensor(
[[ 0.8045827   0.4769051  -0.7812124  -0.996891  ]
 [ 0.33149976 -0.5445254   1.5222508   0.59303206]
 [-0.63509274  0.3703566  -1.0939722  -0.46014452]
 [ 1.5420506  -0.16822556 -0.4390865  -0.4129243 ]], shape=(4, 4), dtype=float32) tf.Tensor(
[[ 1.5420506  -0.16822556 -0.4390865  -0.4129243 ]
 [-0.63509274  0.3703566  -1.0939722  -0.46014452]
 [

### Other ways to make tensors

In [ ]:
# Create a tensor of all ones
tf.ones([10,7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [ ]:
# Create a tensor of zeros
tf.zeros(shape=[10,7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

### Turn NumPy arrays into tensors

The main difference between NumPy arrays and TensorFlow arrays is that tensors can be run on a GPU



In [ ]:
# You can also turn NumPy arrays into tensors
import numpy as np

numpy_A = np.arange(1,25, dtype=np.int32) # create a Numpy array between 1 and 25
numpy_A

# X = tf.constant(some_matrix) # capital for matrix or tensor
# y = tf.constant(vector) # non-capital for vector

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [ ]:
# Now we convert a numpy array to a tensor

A = tf.constant(numpy_A)

In [ ]:
# Modifying the shape of a tensor
B = tf.constant(numpy_A, shape=(2,3,4))
A,B

(<tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>,
 <tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>)

If we want to reshape a tensor, the new elements must add up to give the same number of elements of the original tensor.

In [ ]:
C = tf.constant(numpy_A, shape=(12,2))
C

<tf.Tensor: shape=(12, 2), dtype=int32, numpy=
array([[ 1,  2],
       [ 3,  4],
       [ 5,  6],
       [ 7,  8],
       [ 9, 10],
       [11, 12],
       [13, 14],
       [15, 16],
       [17, 18],
       [19, 20],
       [21, 22],
       [23, 24]], dtype=int32)>

### Getting information from tensors (tensor attributes)

When dealing with tensors, we want to be aware of the following:

* Shape: Length (nº of elements) of each dimension of a tensor. Code: `tensor.shape`
* Rank: The number of tensor dimensions. A scalar has rank 0, a vector has rank 1, a matrix has rank 2, a tensor has rank n. 
Code: `tensor.ndim`
* Axis or dimension: A particular dimension of a tensor. 
Code: `tensor[0]`, `tensor[:,1]`, ...
* Size: The total number of items in the tensor. Code: `tf.size(tensor)`

In [ ]:
# Let's create a rank 4 tensor (4 dimensions)

rank_4_tensor = tf.zeros(shape=[2,3,4,5])
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [ ]:
rank_4_tensor[0]

<tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32)>

In [ ]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [ ]:
# Get various attributes of our tensor

print("Datatype of every element: ", rank_4_tensor.dtype)
print("Number of dimensions (rank): ", rank_4_tensor.ndim)
print("Shape of tensor: ", rank_4_tensor.shape)
print("Elements along the 0 axis: ", rank_4_tensor.shape[0])
print("Elements along the last axis: ", rank_4_tensor.shape[-1])
print("Total number of elements in our tensor: ", tf.size(rank_4_tensor).numpy())

Datatype of every element:  <dtype: 'float32'>
Number of dimensions (rank):  4
Shape of tensor:  (2, 3, 4, 5)
Elements along the 0 axis:  2
Elements along the last axis:  5
Total number of elements in our tensor:  120


### Indexing tensors

Tensors can be indexed just like Python lists

In [ ]:
# Get the first 2 elements of each dimension

rank_4_tensor[:2,:2,:2,:2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [ ]:
# Get the first element from each dimension from each index except for the final one

rank_4_tensor[:1, :, :1, :-1]

<tf.Tensor: shape=(1, 3, 1, 4), dtype=float32, numpy=
array([[[[0., 0., 0., 0.]],

        [[0., 0., 0., 0.]],

        [[0., 0., 0., 0.]]]], dtype=float32)>

In [ ]:
# Create a rank 2 tensor (2 dimensions)

rank_2_tensor = tf.constant([[2, 3, 4],
                             [5, 6, 7]])
rank_2_tensor.ndim


2

In [ ]:
# Get the last item of each of our rank 2 tensor

rank_2_tensor[:,-1].numpy()

array([4, 7], dtype=int32)

In [ ]:
# Add in extra dimension to our rank 2 tensor

rank_3_tensor = rank_2_tensor[..., tf.newaxis] # 
rank_3_tensor

<tf.Tensor: shape=(2, 3, 1), dtype=int32, numpy=
array([[[2],
        [3],
        [4]],

       [[5],
        [6],
        [7]]], dtype=int32)>

Notice the code above when adding a new dimension. The `...` means
every axis till `tf.newaxis`.

In [ ]:
# Alternative to tf.newaxis

tf.expand_dims(rank_2_tensor, axis=-1)

<tf.Tensor: shape=(2, 3, 1), dtype=int32, numpy=
array([[[2],
        [3],
        [4]],

       [[5],
        [6],
        [7]]], dtype=int32)>

In [ ]:
print(rank_2_tensor.shape)
tf.expand_dims(rank_2_tensor, axis=0)

(2, 3)


<tf.Tensor: shape=(1, 2, 3), dtype=int32, numpy=
array([[[2, 3, 4],
        [5, 6, 7]]], dtype=int32)>

### Manipulating tensors (tensor operations)

Pattern discoery involves tensor operations that are essential for training deep learning architectures.

**Basic operations**

`+`, `-`, `*`, `/`


In [ ]:
# Adding values to a tensor using the addition operator

tensor = tf.constant([[1, 2],
                      [3, 4]])

We can also use tensorflow functions for operators. When using these kind of operations, the benefit is that we are making use of TensorFlow's speeding benefits through GPU usage.

In [ ]:
tf.multiply(tensor,10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10, 20],
       [30, 40]], dtype=int32)>

**Matrix Multiplication**

In machine learning, matrix multiplication is one of the most common tensor operations. Matrix multiplication is performed by the dot product. It is not a commutative property of matrices. The elements within an observation of M1 are multiplied by the elements within a column of M2 and each result is added. This implies that if M1(3x2)  M2(3x3) cannot be multiplied.
Instead, having M2(2x3) allows the dot product because the number of columns in M1 matches the number of rows of M2. 

**Cool tip**

https://matrixmultiplication.xyz




In [ ]:
# Tensor multiplication with matmul
tf.matmul(tensor,tensor)

# Don mistake it with tensor * tensor. This multiplication is element wise. Matmul does dot product

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 7, 10],
       [15, 22]], dtype=int32)>

In [ ]:
mat1 = tf.constant([[1, 2, 5],
                    [7, 2, 1],
                    [3, 3, 3]])

mat2 = tf.constant([[3, 5],
                     [6, 7],
                     [1, 8]])

tf.matmul(mat1,mat2)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[20, 59],
       [34, 57],
       [30, 60]], dtype=int32)>

In [ ]:
# Matrix multiplication with Python operator "@"

tensor @ tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 7, 10],
       [15, 22]], dtype=int32)>

In [ ]:

t1 = tf.constant([[1, 2],
                  [3, 4],
                  [5, 6]])

t2 = tf.constant([[7 ,8, 9],
                  [10, 11, 12],
                  [13, 14, 15]])


# Multiplication of these two tensors work by either multiplying t2 x t1, or transposing t1.
tf.matmul(t2,t1), tf.matmul(tf.transpose(t1),t2)

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 76, 100],
        [103, 136],
        [130, 172]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[102, 111, 120],
        [132, 144, 156]], dtype=int32)>)

**The dot product**
Matrix multiplication is also referred to as the dot product.

We can perform matrix multiplication using:
* `tf.matmul`
* `tf.tensordot`

In [ ]:
# Perform the dot product on X and Y (requires X or Y to be transposed)

tf.tensordot(tf.transpose(t1),t2, axes=1)

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[102, 111, 120],
       [132, 144, 156]], dtype=int32)>

With `tf.tensordot` we sum the product of elements from matrices t1 and t2 over the indices specified by axes.

* Example 1: When `t1` and `t2` are matrices (order 2), the case `axes = 1` is equivalent to matrix multiplication.
* Example 2: when `t1` and `t2` are matrices (order 2), the case `axes = [[1], [0]]` is quivalent to matrix multiplication.
* Example 3: When `t1` and `t2` are matrices (order 2), the case `axes=0` gives the outer product, a tensor of order 4.

In [ ]:
# Let's change the shape of t1:

t1 = tf.reshape(t1, shape = (2,3))
print(t1.shape,t2.shape)

tf.matmul(t1,t2)

(2, 3) (3, 3)


<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 66,  72,  78],
       [156, 171, 186]], dtype=int32)>

Here the result works but it is misleading because reshaping the tensor is not the same result as transpose. Do not be mislead by this silent error. The code will work but the results will be wrong when the right operation should be transpose.

**Nonetheless, we will be working a lot with reshaping. Just to be aware!**

**The difference between transpose and reshape is that transposing a tensor flips the axes and reshaping just squeezes the values to fit the output shape**

In [ ]:
# Perform matrix multiplication between X and Y (Reshaped)

X = tf.constant([[2, 4, 6],
                 [1, 3, 5],
                 [8, 10, 12]])
Y = tf.constant([[6, 8, 10],
                 [11, 13, 15]])

print(X.shape,Y.shape)
 

tf.matmul(X, tf.reshape(Y, shape=(3,2))), tf.matmul(X,tf.transpose(Y))

(3, 3) (2, 3)


(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[130, 150],
        [101, 116],
        [304, 354]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[104, 164],
        [ 80, 125],
        [248, 398]], dtype=int32)>)

>Observe again obtaining different results between reshape and transpose. If done with a tensor of ones, you will get the same value, but it is misleading!

> Generally when performing matrix multiplication on two tensors and one of the axes doesn't line up, we will rather transpose one of the tensors to satisfy the dot product rules.


### Change the datatype of a tensor

In [ ]:
# Create a new tensor with default datatype (float32)

B = tf.constant([1.7, 7.4])
B.dtype

tf.float32

In [ ]:
C = tf.constant([7, 10])
C.dtype

tf.int32

When we read the number after int or float data types, we are referring to the number of bits, not bytes! Careful there. One bit is the basic unit of data and it is 1 or 0. It takes 4 bits to represent any digit.

Tensorflow can work iwth mixed precision. This means the use of both 16-bit and 32-bit floating-point types in a model during training to make it run faster and use less memory. the model will have a lower step time and train equally as well in terms of evaluation metrics such as accuracy. 

Today, most models use the float32 dtype, which takes 32 bits of memory. However there are two lower'precision dtypes, float16 and bfloat16, each which take 16 bits of memory instead. Modern accelerators can run operations faster in the 16-bit dtypes, as they have specialized hardware to run 16-bit computations and 16-bit dtypes can be read from memory faster.

NVIDIA GPUs can run operations in float16 faster than in float32, and TPUs can run operations in bfloat16 faster than float32. Therefore, these lower-precision dtypes should be used wheneer possible on those devices. However, variables and a few computations should still be in float32 for numeric reasons so that the model trains to the same quality. The Keras mixed precision API allows you to use a mix of either float16 or bfloat16 with float32, to get the performance benefits from float16/bfloat16 and the numeric stability benefits from float32.

In [ ]:
# Change from float32 to float16. This is called reduced precision

B = tf.cast(B, dtype=tf.float16)
B

<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 7.4], dtype=float16)>

We have reduced the amount of bits from our tensor from 32 to 16. This now may be meaningless, but when we are working with large datasets, this may save our butts from running out of memory to make calculations potentially twice as fast

In [ ]:
# Change from int32 to float32

E = tf.cast(C, dtype=tf.float32)

In [ ]:
E_float16 = tf.cast(E, dtype=tf.float16)
E,E_float16

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 7., 10.], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float16, numpy=array([ 7., 10.], dtype=float16)>)


### Aggregating tensors

Aggregating tensors = condensing them from multiple values down to a smaller amount of values. 


In [ ]:
# Get the absolute vaues 

D = tf.constant([-6, -9])
D

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([-6, -9], dtype=int32)>

In [ ]:
# Get the absolute value

tf.abs(D)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([6, 9], dtype=int32)>

We go through the following form of aggregation:
* Get minimum
* Get maximum
* Get the mean of a tensor
* Get the sum of a tensor 


In [ ]:
# Create a random tensor with values between 0 and 100 of size 50

E = tf.constant(np.random.randint(0,100,size=50))
E

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([86, 95, 75, 85, 77, 14, 48, 76, 62, 75, 28, 89, 82,  8, 13, 35, 55,
       83, 60, 28, 18, 54, 78, 21, 36, 19, 97,  4, 84, 75, 48, 54, 96, 32,
        9, 28, 86, 92, 64,  7, 16, 37, 11, 65, 25, 56, 44, 92, 38,  5])>

In [ ]:
tf.size(E), E.shape, E.ndim

(<tf.Tensor: shape=(), dtype=int32, numpy=50>, TensorShape([50]), 1)

In [ ]:
# Find the minimum

tf.reduce_min(E)

<tf.Tensor: shape=(), dtype=int64, numpy=4>

In [ ]:
# find the maximum

tf.reduce_max(E)

<tf.Tensor: shape=(), dtype=int64, numpy=97>

In [ ]:
# Get the maximum

tf.reduce_mean(E)

<tf.Tensor: shape=(), dtype=int64, numpy=51>

In [ ]:
# get the sum

tf.reduce_sum(E)

<tf.Tensor: shape=(), dtype=int64, numpy=2565>

Now we will find the variance and std of our `E` tensor, using our tensorflow methods

In [ ]:
# Find the variance

E_mean = tf.reduce_mean(E)
E_var = tf.reduce_sum((E - E_mean)**2)/E.shape[0]
E_std = np.sqrt(E_var)
E_std

29.66782769263702

In [ ]:
# To find the variance of our tensor, we need access to tensorflow_probability
import tensorflow_probability as tfp

tfp.stats.variance(E)
np.sqrt(tfp.stats.variance(E))


29.664793948382652

Remember we want to use TensorFlow functions and features the most we can so we make use of the GPU acceleration. 

**Note:**
To find descriptive statistics like STD we need to have our data as float instead of int.

In [ ]:
E = tf.cast(E, dtype=tf.float32)
E
tf.math.reduce_std(E)

<tf.Tensor: shape=(), dtype=float32, numpy=29.666311>

In [ ]:
# An easy way to find the variance of a tensor only with tensorflow

tf.math.reduce_variance(tf.cast(E, dtype=tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=880.09>

### Find the positional maximum and minimum

In many cases on classification problems, our output is a probability and the highest probability is assigned to the category to which the observation belongs in the classification. To make the neural network output understandable to us humans we need to use ways to find the positional maximum from the output.

In [ ]:
# Create a new tensor for finding positional minimum and maximum

tf.random.set_seed(42)
F = tf.random.uniform(shape=[50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [ ]:
# Index our largest value position
F[tf.argmax(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [ ]:
# Find largest value
tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [ ]:
# Check for equality
F[tf.argmax(F)] == tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [ ]:
# Find the positional minimum
tf.argmin(F)

<tf.Tensor: shape=(), dtype=int64, numpy=16>

In [ ]:
# Find the minimum using positional minimum index

F[tf.argmin(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.009463668>

### Squeezing a tensor (removing all single dimensions)


In [ ]:
# Create a tensor to get started
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape=[50]), shape = (1, 1, 1, 1, 50))
G

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
           0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
           0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
           0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
           0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
           0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
           0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
           0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
           0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
           0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043]]]]],
      dtype=float32)>

In [ ]:
G.shape

TensorShape([1, 1, 1, 1, 50])

In [ ]:
# Squeeze will remove dimensions of shape 1 from the size of the tensor
G_squeezed = tf.squeeze(G)
G_squeezed

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

### One-hot encoding tensors



In [ ]:
# Create a list of indices
some_list = [0, 1, 2, 3] # could be red, green, blue, purple

# one hot encode our list of indices
tf.one_hot(some_list, 4)


<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

### Additional math operations with tensors

In [ ]:
# Create a new tensor
H = tf.range(1,10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [ ]:
# Square it
tf.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>

In [ ]:
# Find the square root (method requires non-int type)
tf.sqrt(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.       , 1.4142135, 1.7320508, 2.       , 2.2360678, 2.4494896,
       2.6457512, 2.828427 , 3.       ], dtype=float32)>

In [ ]:
# find the log
tf.math.log(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

### Test three different math functions with tensorflow

These functions are all activation functions that are commonly used in neural networks. Here we provide a small sample of them.

* tanh(z): Hyperbolic tangent. `[(exp(z) - exp(-z)] / [exp(z) + exp(-z)]`. Its input range is `[-inf, inf]` and its output range is `[-1,1]`

* Sigmoid(x) = `1 / (1 + exp(-x))`. Its input range is `[-inf,inf]` and its output range is `[0,1]`. The sigmoid function is also called logistic function, and it plays a fundamental role in logistic regression.

* softplus(y) = `log(1 + exp(y))`. It is a smooth approximation of `ReLu` activation function, and sometimes it is used in place of it. 

In [ ]:
# tanh (hyperbolic tangent)
tf.math.tanh(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.7615942 , 0.9640276 , 0.9950547 , 0.9993292 , 0.99990916,
       0.9999876 , 0.99999833, 1.        , 1.        ], dtype=float32)>

In [ ]:
# sigmoid
tf.math.sigmoid(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.73105854, 0.880797  , 0.95257413, 0.98201376, 0.9933072 ,
       0.9975274 , 0.999089  , 0.99966466, 0.9998766 ], dtype=float32)>

In [ ]:
# softplus
tf.math.softplus(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.3132616, 2.126928 , 3.0485873, 4.01815  , 5.0067153, 6.0024757,
       7.0009117, 8.000336 , 9.000123 ], dtype=float32)>

### Tensors and NumPy

TensorFlow interacts beautifully with NumPy arrays.

The beautiful part of it is some of the functionality that may not work with our datatype, we can convert it to numpy.

In [ ]:
# create a tensor directly from a numpy array
J = tf.constant(np.array([3.,7.,10.]), dtype=tf.float32)
J

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 3.,  7., 10.], dtype=float32)>

In [ ]:
# Convert our tensor back to a numpy array
np.array(J),type(np.array(J))

(array([ 3.,  7., 10.], dtype=float32), numpy.ndarray)

In [ ]:
# Convert tensor J to a numpy array
J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.], dtype=float32), numpy.ndarray)

In [ ]:
J = tf.constant([3.])
J.numpy()

array([3.], dtype=float32)

In [ ]:
# The default types of each are slightly different

numpy_J = tf.constant(np.array([3.,7.,10.]))
tensor_J = tf.constant([3.,7.,10.])

# Check data types of each
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

Be aware of this. One slight difference between creating tensorflows straight from python lists and numpy arrays, is that the latter will have a data type of `tf.float64`, while the first one is `tf.float32`
Remember the default data type for Tensor flow is `tf.float32`.

### How to find access to GPUs

*   With google colab, go to Runtime > Change runtime type



In [ ]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
!nvidia-smi

Fri Jun 10 15:54:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    29W /  70W |    266MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**Note** If you have access to a CUDA-enable GPU, TensorFlow will automatically make use of it